In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
with open("options.txt", 'r') as f:
    options = f.readlines()
    options = {option.split("=")[0]: option.split("=")[1].strip() for option in options}
print(options)

{'hanoi_scenario_dir': 'C:\\Users\\mjnst\\Desktop\\Thesis\\Hanoi_CMH\\Scenario-1', 'RUG_dir': 'C:\\Users\\mjnst\\Desktop\\Thesis\\RUG_data_5years', 'RUG_raw_csv': 'C:\\Users\\mjnst\\Desktop\\Thesis\\rug_csv.csv', 'RUG_timeseries': 'C:\\Users\\mjnst\\Desktop\\Thesis\\rug_timeseries.pkl'}


In [3]:
RUG = pd.read_pickle(options["RUG_timeseries"])
RUG = RUG.filter(['Oosterbeek-hoog\Fletcher BV|Supplied'])
RUG=RUG.head(int(525600/2))
RUG

,Oosterbeek-hoog\Fletcher BV|Supplied
date,
2017-01-01 00:00:00+01:00,1.03
2017-01-01 00:01:00+01:00,1.03
2017-01-01 00:02:00+01:00,1.03
2017-01-01 00:03:00+01:00,1.03
2017-01-01 00:04:00+01:00,1.03
...,...
2017-07-02 11:55:00+01:00,0.42
2017-07-02 11:56:00+01:00,0.42
2017-07-02 11:57:00+01:00,0.42


In [4]:
# fix random seed for reproducibility
tf.random.set_seed(7)

In [5]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(RUG)

In [6]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

176076 86724


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [9]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [10]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [11]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [12]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=8, batch_size=1, verbose=2)

Epoch 1/100
176074/176074 - 776s - loss: 1.4561e-04 - accuracy: 6.8153e-04 - 776s/epoch - 4ms/step
Epoch 2/100
176074/176074 - 734s - loss: 1.0543e-04 - accuracy: 6.8153e-04 - 734s/epoch - 4ms/step
Epoch 3/100
176074/176074 - 756s - loss: 1.0516e-04 - accuracy: 6.8153e-04 - 756s/epoch - 4ms/step
Epoch 4/100
176074/176074 - 799s - loss: 1.0157e-04 - accuracy: 6.8153e-04 - 799s/epoch - 5ms/step
Epoch 5/100
176074/176074 - 853s - loss: 9.9425e-05 - accuracy: 6.8153e-04 - 853s/epoch - 5ms/step
Epoch 6/100
176074/176074 - 888s - loss: 9.8796e-05 - accuracy: 6.8153e-04 - 888s/epoch - 5ms/step
Epoch 7/100
176074/176074 - 958s - loss: 9.8354e-05 - accuracy: 6.8153e-04 - 958s/epoch - 5ms/step
Epoch 8/100
176074/176074 - 1126s - loss: 9.7977e-05 - accuracy: 6.8153e-04 - 1126s/epoch - 6ms/step
Epoch 9/100
176074/176074 - 1147s - loss: 9.7672e-05 - accuracy: 6.8153e-04 - 1147s/epoch - 7ms/step
Epoch 10/100
176074/176074 - 1128s - loss: 9.7453e-05 - accuracy: 6.8153e-04 - 1128s/epoch - 6ms/step
Epo

KeyboardInterrupt: 

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))